In [16]:
import os

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3

from notebooks import fetch_all_pools_info
from notebooks import get_abi

load_dotenv()
web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
ARBITRUM_CHAIN_LITERAL = "ARBITRUM"
ARBITRUM_TOKEN_ADDR = Web3.to_checksum_address("0x912ce59144191c1204e64559fe8253a0e49e6548")
# Fetch all pools from Balancer API
all_pools = fetch_all_pools_info("arbitrum")

# Collect arb gauges
arb_gauges = {}
for pool in all_pools:
    # Only collect gauges for the arb chain and that are not killed
    if pool['chain'] == ARBITRUM_CHAIN_LITERAL and pool['gauge']['isKilled'] is False:
        _gauge_addr = Web3.to_checksum_address(pool['gauge']['address'])
        arb_gauges[_gauge_addr] = {
            'gaugeAddress': pool['gauge']['address'],
            'pool': pool['address'],
            'symbol': pool['symbol'],
            'id': pool['id'],
            'childGauge': Web3.to_checksum_address(pool['gauge']['childGaugeAddress']),
        }
print(f"Found {len(arb_gauges)} gauges on Arbitrum")

# Now go through all gauges and check that each of them has reward tokens set to ARB
gauge_reward_data = {}
for _, pool_info in arb_gauges.items():
    child_gauge_contract = web3.eth.contract(address=pool_info['childGauge'],
                                             abi=get_abi("ChildGauge"))
    data = child_gauge_contract.functions.reward_data(ARBITRUM_TOKEN_ADDR).call()
    gauge_reward_data[pool_info['childGauge']] = {
        'pool_id': pool_info['id'],
        'symbol': pool_info['symbol'],
        'root_gauge': pool_info['gaugeAddress'],
        'child_gauge': pool_info['childGauge'],
        'distributor': data[0],
        'period_finish': data[1],
        'rate': data[2],
        'last_update': data[3],
        'integral': data[4],
    }

# Into a dataframe and print
df = pd.DataFrame.from_dict(gauge_reward_data, orient='index')
df.to_csv('arb_gauge_reward_data.csv', index=False)

Found 27 gauges on Arbitrum
